In [1]:
!pip install -q pandas_ta yfinance seaborn

You should consider upgrading via the 'd:\kaushik\github repos\time_series_analysis_and_forecasting_using_neural_networks\swing_trading_strategy\env\scripts\python.exe -m pip install --upgrade pip' command.


In [44]:
import yfinance as yf
import numpy as np
import pandas_ta as ta
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

In [45]:
def trade_donchian(row):
    global trades, trade_open
    row = row.to_dict()

    if((trade_open==True) and (row['long'] == 1)): pass
    elif((trade_open==False) and (row['long'] == 1)):
        # open trade
        trade_open = True
        _trade = {
            'buy_date': row['next_date'],
            'buy_price': round(row['next_day_open_price']*1.005,2),
            'sell_price': None,
            'sell_date': None,
        }
        trades.append(_trade)
        del _trade
    elif((trade_open==False) and (row['short'] == 1)): pass
    elif((trade_open==True) and (row['short'] == 1)):
        # close trade
        trade_open = False
        _trade = trades[-1]
        _trade['sell_date'] = row['next_date']
        _trade['sell_price'] = round(row['next_day_open_price']*0.995,2)
        trades[-1] = _trade
        del _trade

In [46]:
def trade_donchian(row):
    global trades, trade_open
    row = row.to_dict()

    if((trade_open==True) and (row['long'] == 1)):
        _trade = trades[-1]
        if(float(_trade['swing_low'])>float(row['Close'])): _trade['swing_low'] = row['Close']
        if(float(_trade['swing_low'])>float(row['Adj Close'])): _trade['swing_low'] = row['Adj Close']
        if(float(_trade['swing_high'])<float(row['Close'])): _trade['swing_high'] = row['Close']
        if(float(_trade['swing_high'])<float(row['Adj Close'])): _trade['swing_high'] = row['Adj Close']
        trades[-1] = _trade
        del _trade
        
    elif((trade_open==False) and (row['long'] == 1)):
        # open trade
        trade_open = True
        _trade = {
            'buy_date': row['next_date'],
            'swing_high': round(row['next_day_open_price']*1.005,2),
            'swing_low': round(row['next_day_open_price']*1.005,2),
            'buy_price': round(row['next_day_open_price']*1.005,2),
            'sell_price': None,
            'sell_date': None,
        }
        trades.append(_trade)
        del _trade
        
    elif((trade_open==False) and (row['short'] == 1)): pass
        
    elif((trade_open==True) and (row['short'] == 1)):
        # close trade
        trade_open = False
        _trade = trades[-1]
        _trade['sell_date'] = row['next_date']
        _trade['sell_price'] = round(row['next_day_open_price']*0.995,2)

        if(float(_trade['swing_low'])>float(row['Close'])): _trade['swing_low'] = row['Close']
        if(float(_trade['swing_low'])>float(row['Adj Close'])): _trade['swing_low'] = row['Adj Close']
        if(float(_trade['swing_high'])<float(row['Close'])): _trade['swing_high'] = row['Close']
        if(float(_trade['swing_high'])<float(row['Adj Close'])): _trade['swing_high'] = row['Adj Close']
            
        trades[-1] = _trade
        del _trade

In [26]:
# data = yf.download(f'hcltech.ns', period='10y', progress=False).reset_index()

In [27]:
# data['week_day'] = pd.to_datetime(data['Date']).dt.day_name()

In [28]:
# data[~data['week_day'].isin(['Saturday', 'Sunday'])]

In [29]:
def backtest(stock, period, low=20, high=20):
    global trades, trade_open
    
    data = yf.download(f'{stock}.ns', period=period, progress=False).reset_index()
    
    data['week_day'] = pd.to_datetime(data['Date']).dt.day_name()
    
    data = data[~data['week_day'].isin(['Saturday', 'Sunday'])]
    
    data[['low', 'mid', 'high']] = data.ta.donchian(lower_length=low, upper_length=high)
    
    data['long'] = ((data['Close']==data['low'])|(data['Low']==data['low'])).astype('int')
    data['short'] = ((data['Close']==data['high'])|(data['High']==data['high'])).astype('int')
    
    data['next_day_open_price'] = data['Open'].shift(-1)
    data['next_date'] = data['Date'].shift(-1).astype('string')
    
    trade_open = False
    
    trades = []
    
    data.dropna(inplace=True)
    
    cols = ['Date', 'Open', 'Close', 'Adj Close', 'Low', 'High', 'low', 'mid', 'high', 'long', 'short', 'next_day_open_price', 'next_date']
    data = data[cols]
    data.apply(trade_donchian, axis=1)

    if(len(trades)==0): return None
    x = pd.DataFrame(trades)
    
    x['buy_date'] = pd.to_datetime(x['buy_date'], format="%Y-%m-%d", dayfirst=True)
    x['sell_date'] = pd.to_datetime(x['sell_date'], format="%Y-%m-%d", dayfirst=True)
    x['returns'] = round(100*(x['sell_price']-x['buy_price'])/x['buy_price'],2)
    x['holding_period'] = (x['sell_date'] - x['buy_date']).dt.days
    x['stock'] = stock
    return x

In [47]:
TRADES = pd.DataFrame()
trades = []
trade_open = False

file_name = 'n500'

stocks_list = [i.upper() for i in pd.read_csv(f'./index_stocks/{file_name}.csv')['Symbol'].values if 'adani' not in i.lower()]
# stocks_list = ['NIFTYBEES', 'UTIBANKETF', 'BANKBEES']

for stock in stocks_list:
    _tr = backtest(stock, '10y', 20, 20)
    if(len(TRADES)==0): TRADES = _tr
    else: TRADES = pd.concat([TRADES, _tr], ignore_index=True)

In [48]:
TRADES.to_csv(f'etfs.csv', index=False)

In [41]:
TRADES.to_csv(f'{file_name}trades.csv', index=False)

In [43]:
for file_name in ['n50','n100','n200','n500']:
    TRADES = pd.DataFrame()
    trades = []
    trade_open = False
    
    stocks_list = [i.upper() for i in pd.read_csv(f'./index_stocks/{file_name}.csv')['Symbol'].values if 'adani' not in i.lower()]
    
    for stock in stocks_list:
        _tr = backtest(stock, '10y', 20, 20)
        if(len(TRADES)==0): TRADES = _tr
        else: TRADES = pd.concat([TRADES, _tr], ignore_index=True)

    TRADES.to_csv(f'{file_name}trades.csv', index=False)

In [10]:
historical_trades = TRADES.dropna(how='any')

In [11]:
historical_trades[['returns', 'holding_period']].describe()

,returns,holding_period
count,16192.000000,16192.000000
mean,1.216350,39.719615
std,10.840928,30.654716
min,-87.210000,1.000000
25%,-2.862500,19.000000
50%,2.930000,32.000000
75%,7.200000,54.000000
max,146.150000,839.000000


In [12]:
historical_trades.loc[:,'pos_neg'] = (historical_trades['returns']>0).astype('int')

C:\Users\alarv\AppData\Local\Temp\ipykernel_12944\29694905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_trades.loc[:,'pos_neg'] = (historical_trades['returns']>0).astype('int')


In [13]:
historical_trades['pos_neg'].value_counts(normalize=True)*100

pos_neg
1    65.062994
0    34.937006
Name: proportion, dtype: float64

In [14]:
historical_trades.groupby('pos_neg')[['returns', 'holding_period']].describe().T

# historical_trades.groupby('pos_neg')[['holding_period']].describe()

pos_neg                         0             1
returns        count  5657.000000  10535.000000
               mean     -9.435593      6.936145
               std      10.147189      5.630525
               min     -87.210000      0.010000
               25%     -12.870000      3.120000
               50%      -6.120000      5.760000
               75%      -2.440000      9.350000
               max       0.000000    146.150000
holding_period count  5657.000000  10535.000000
               mean     66.170762     25.516089
               std      31.830680     17.968099
               min       1.000000      1.000000
               25%      44.000000     14.000000
               50%      61.000000     23.000000
               75%      80.000000     34.000000
               max     240.000000    839.000000

In [15]:
historical_trades.groupby('pos_neg')[['returns']].sum()

,returns
pos_neg,
0,-53377.15
1,73072.29


In [16]:
historical_trades[['returns']].sum()

returns    19695.14
dtype: float64

In [17]:
historical_trades.groupby(pd.cut(historical_trades['returns'], [-10000,-10,-5,0,5,10,15,20,10000]))[['returns', 'holding_period']].describe().T

returns               (-10000, -10]    (-10, -5]      (-5, 0]       (0, 5]  \
returns        count    1893.000000  1297.000000  2467.000000  4594.000000   
               mean      -20.348325    -7.284665    -2.192769     2.687410   
               std        10.628159     1.443673     1.439027     1.400112   
               min       -87.210000    -9.990000    -4.990000     0.010000   
               25%       -24.260000    -8.460000    -3.400000     1.510000   
               50%       -16.770000    -7.220000    -2.050000     2.800000   
               75%       -12.830000    -6.000000    -0.915000     3.890000   
               max       -10.000000    -5.000000     0.000000     5.000000   
holding_period count    1893.000000  1297.000000  2467.000000  4594.000000   
               mean       89.475436    65.540478    48.619781    30.297562   
               std        34.455767    22.952012    20.269488    17.828001   
               min         1.000000     1.000000     1.000000     1.000000   
               25%        64.000000    49.000000    36.000000    18.000000   
               50%        82.000000    62.000000    47.000000    28.000000   
               75%       108.000000    77.000000    60.000000    40.000000   
               max       240.000000   194.000000   169.000000   128.000000   

returns                   (5, 10]     (10, 15]    (15, 20]  (20, 10000]  
returns        count  3645.000000  1515.000000  487.000000   294.000000  
               mean      7.243402    12.031109   17.091622    26.440068  
               std       1.413533     1.405407    1.380679     9.964096  
               min       5.010000    10.010000   15.020000    20.010000  
               25%       6.020000    10.800000   15.890000    21.627500  
               50%       7.120000    11.860000   16.930000    23.840000  
               75%       8.430000    13.150000   18.165000    28.027500  
               max      10.000000    15.000000   19.970000   146.150000  
holding_period count  3645.000000  1515.000000  487.000000   294.000000  
               mean     22.547599    20.741254   20.626283    20.309524  
               std      19.454660    12.724148   12.128596    13.574071  
               min       1.000000     1.000000    1.000000     1.000000  
               25%      12.000000    12.000000   13.000000    12.000000  
               50%      20.000000    19.000000   19.000000    18.000000  
               75%      29.000000    27.000000   25.000000    25.750000  
               max     839.000000   135.000000   77.000000   100.000000

In [18]:
x = historical_trades.groupby(pd.cut(historical_trades['returns'], [-10000,-10,-5,0,5,10,15,20,10000]))[['returns', 'holding_period']].describe()['returns']['count'].reset_index()
x['percent'] = 100*x['count']/x['count'].sum()
x

,returns,count,percent
0,"(-10000, -10]",1893.0,11.690958
1,"(-10, -5]",1297.0,8.010128
2,"(-5, 0]",2467.0,15.235919
3,"(0, 5]",4594.0,28.372036
4,"(5, 10]",3645.0,22.511117
5,"(10, 15]",1515.0,9.356472
6,"(15, 20]",487.0,3.007658
7,"(20, 10000]",294.0,1.815711
